In [1]:
import numpy as np
import numba

import emcee
import pystan

import bebi103

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
n_dim = 2
mu = np.random.uniform(low=-1.0, high=1.0, size=n_dim)
sigma = np.random.uniform(low=0.01, high=1.0, size=n_dim)

In [4]:
def log_prob(x, mu, sigma):
    return (-np.sum(np.log(2*np.pi*sigma)) / 2 
                - np.sum((x-mu)**2 / 2 / sigma**2))

In [5]:
# Get samples out of distribution
true_samples = np.random.multivariate_normal(mu, np.diag(sigma**2), size=5000)

true_lnprob = np.array([log_prob(x, mu, sigma) for x in true_samples])

In [6]:
n_walkers = 50   # number of MCMC walkers
n_burn = 1000    # "burn-in" period to let chains stabilize
n_steps = 1000   # number of MCMC steps to take after burn-in

# x0[i,j] is the starting point for walk i along variable j.
x0 = np.empty((n_walkers, n_dim))
x0 = np.random.uniform(low=-1.0, high=1.0, size=(n_walkers, n_dim))

In [7]:
sampler = emcee.EnsembleSampler(n_walkers, n_dim, log_prob, 
                                args=(mu, sigma), threads=6)

# Do burn-in
pos, prob, state = sampler.run_mcmc(x0, n_burn, storechain=False)

# Sample again, starting from end burn-in state
_ = sampler.run_mcmc(pos, n_steps)

In [8]:
p = bebi103.viz.ecdf(true_lnprob)
p = bebi103.viz.ecdf(sampler.flatlnprobability, formal=True, color='orange', p=p)
bokeh.io.show(p)

In [2]:
model_code = """
data {
  int N;
  real mu[N];
  real sigma[N];
}


parameters {
  real x[N];
}


model {
  x ~ normal(mu, sigma);
}
"""

sm = bebi103.stan.StanModel(model_code=model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_ac358ff473f2097d895e084b9885e2f7 NOW.
/Users/Justin/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /var/folders/y7/r162f1fd0zv1vzxkb5nktz300000gn/T/tmpruks730w/stanfit4anon_model_ac358ff473f2097d895e084b9885e2f7_8304010235845583897.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


CompileError: command 'clang' failed with exit status 1

In [10]:
pystan.__version__

'2.17.1.0'